In [4]:
import pandas as pd
from datetime import datetime
from glob import glob 
files = glob("/dccstor/srosent2/generative/appen/round1_jobs/*json.zip") #["job_1999101.json","job_1988758.json", "job_2004889.json","job_2006984.json", "job_2022794.json", "job_2035917.json"]

seen_ids = set()

for file_name in files:
    # file_name = "/dccstor/srosent2/generative/appen/round1_jobs/" + file
    data = pd.read_json(file_name, lines=True)

    for index, row in data.iterrows():
        info = row['results']['judgments'][0]
        # print(row['results'])
            
        seen_ids.add(info['unit_data']['question_id'])

print(len(seen_ids))
# print(seen_ids)

12570


In [1]:
# load tydi json
import glob
import gzip
import json
import spacy

# spacy.cli.download('en_core_web_sm')

nlp = spacy.load('en_core_web_sm')

train_data_dir = "/dccstor/srosent2/primeqa/data/nq-lfqa-train/*jsonl"
dev_data_dir = "/dccstor/srosent2/primeqa/data/dev/nq-lfqa/*jsonl"

train_files = glob.glob(train_data_dir)
dev_files = glob.glob(dev_data_dir)

def load_json_from_file(gt_file_patterns):
    data = []
    if gt_file_patterns.endswith('gz'):
        f = gzip.open(gt_file_patterns, 'rt', encoding='utf-8')
    else:
        f = open(gt_file_patterns, 'rt', encoding='utf-8')
    lines = f.readlines()
    for line in lines:
        data.append(json.loads(line))
    return data

In [10]:
train_data = []
for train_file in train_files:
    train_data.extend(load_json_from_file(train_file))

In [3]:
dev_data = []
for dev_file in dev_files:
    dev_data.extend(load_json_from_file(dev_file))

print(len(dev_data))

seen = 0
not_la = 0
seen_dev_ids = set()
for example in dev_data:
    
    qtype = example['type'][0]
    
    if qtype != 'la':
        not_la += 1
        continue
    if str(example['example_id']) in seen_ids:
        seen_dev_ids.add(example['example_id'])
        seen += 1
print(not_la)
print(seen)
print(len(seen_dev_ids))

3393


NameError: name 'seen_ids' is not defined

In [15]:
train_files

['/dccstor/srosent2/primeqa/data/nq-lfqa-train/nq-train-tydiformat4.jsonl',
 '/dccstor/srosent2/primeqa/data/nq-lfqa-train/nq-train-tydiformat2.jsonl',
 '/dccstor/srosent2/primeqa/data/nq-lfqa-train/nq-train-tydiformat0.jsonl',
 '/dccstor/srosent2/primeqa/data/nq-lfqa-train/nq-train-tydiformat1.jsonl',
 '/dccstor/srosent2/primeqa/data/nq-lfqa-train/nq-train-tydiformat.jsonl',
 '/dccstor/srosent2/primeqa/data/nq-lfqa-train/nq-train-tydiformat3.jsonl']

In [12]:
len(train_data)

61499

In [7]:
import csv

# part1 == also 2000?
# up to 19505 to get part2 (2000 each)
# up to 30209 to get part3 (2000 each)
# up to 41213 to get part4 (2000 each)
# up to 53351 to get part5 (2000 each)
# 64067 part6 (2000)
# to end part7 64504 (175)

def is_majority_type(types, type):

    count = 0

    for t in types:
        if t == type:
            count += 1
    if count >= 3:
        return True
    return False

def get_appen_data(data, type, exclude_ids, output_file, start):
    f = open(output_file, 'w', encoding='UTF8', newline='')
    writer = csv.writer(f, delimiter='\t')
    writer.writerow(["question_id","question","title","url","long_answer","minimal_text"])

    count = 0
    index = start
    lines = []
    excluded = 0
    for example in data[start:]:
        index += 1
        # if index >= 19505:
        #     continue
        # qtype = example['type'][0]
        
        # if qtype != type:
        #     continue

        if not is_majority_type(example['type'], type):
            continue
        
        example_id = example['example_id']

        if str(example_id) in exclude_ids:
            excluded += 1
            continue

        question = example["question_text"]

        q_word = question.split()[0]

        annotation = example['annotations'][0]

        if annotation == None:
            continue

        minimal_text = ""
        if annotation['minimal_answer']['plaintext_start_byte'] != -1:
            minimal_text = example['document_plaintext'].encode('utf-8')[annotation['minimal_answer']['plaintext_start_byte']:annotation['minimal_answer']['plaintext_end_byte']].decode('utf-8')
            continue
        passage_offsets = example['passage_answer_candidates'][annotation['passage_answer']['candidate_index']]
        passage_text = example['document_plaintext'].encode('utf-8')[passage_offsets['plaintext_start_byte']:passage_offsets['plaintext_end_byte']].decode('utf-8')
        
        passage_sentences = ""
        sentences = nlp(passage_text)
        
        num_sentences = 0
        for sentence in sentences.sents:
            num_sentences += 1
            passage_sentences += sentence.text + "&nbsp;"

        if num_sentences < 5:
            continue

        count += 1
        if count % 100 == 0:
            print(count)
            print(index)
        if count > 2000:
            break

        writer.writerow([str(example_id), question, example['document_title'], example['document_url'], passage_sentences, minimal_text])
    print(count)
    print(excluded)
    f.close()
           

In [11]:
get_appen_data(train_data, "la", seen_ids, "/dccstor/srosent2/generative/data_for_appen/full_nq_task_nomin_train_more.tsv", 0)
# get_appen_data(dev_data, "la", seen_ids, "/dccstor/srosent2/generative/data_for_appen/full_nq_task_nomin_dev.tsv", 0)

0
0


In [2]:
# load final data and split into train and dev
import pandas as pd

annotated_data = pd.read_json("/dccstor/srosent2/generative/appen/NQ_formatted_answered_single-8.28.23.json", dtype={'id':str}, lines=True)
annotated_data.sample(5)

,id,input,passages,output
4835,-860893014778759502,what does the name judas mean in greek,"[{'title': 'Judas Iscariot', 'text': 'The name...",[{'answer': 'The name Judas is a Greek renderi...
1019,-3233216490717507766,where did the name carl jr come from,"[{'title': 'Carl's Jr.', 'text': 'Carl Karcher...",[{'answer': 'The first two Carl 's Jr. restaur...
10126,-7138712513392194276,fetal hematopoiesis occurs in which structure ...,"[{'title': 'Haematopoiesis', 'text': 'In some ...","[{'answer': 'During fetal development , since ..."
6251,2170813232674301260,when did krakow become the capital of poland,"[{'title': 'Kraków', 'text': 'Kraków ( Polish ...",[{'answer': 'Kraków has been the capital of Le...
5299,55946526549723721,what era did the woolly mammoth live in,"[{'title': 'Woolly mammoth', 'text': 'The wool...",[{'answer': 'The woolly mammoth is an extinct ...


In [4]:
dev_ids = []

for item in dev_data:
    dev_ids.append(str(item['example_id']))

print(len(dev_ids))
print(dev_ids[0:5])

3393
['-6570496346595660652', '4111902318448915849', '3599421094587589904', '3367999921663366204', '-2103986527722712835']


In [5]:
dev_annotated_data = annotated_data[annotated_data['id'].isin(dev_ids)]
train_annotated_data = annotated_data[~annotated_data['id'].isin(dev_ids)]

train_annotated_data.to_json("/dccstor/srosent2/generative/appen/NQ_formatted_answered_single-8.28.23_train.json", lines=True, orient='records')
dev_annotated_data.to_json("/dccstor/srosent2/generative/appen/NQ_formatted_answered_single-8.28.23_dev.json", lines=True, orient='records')

print(len(annotated_data))
print(len(dev_annotated_data))
print(len(train_annotated_data))

11941
303
11638


In [16]:
single_annotated_data = annotated_data[annotated_data['id'].isin(dev_ids)]

print(len(single_annotated_data))

303


In [11]:
list(annotated_data['id'][0:20])

['8045984229282682032',
 '8116539286638468011',
 '-3228156810154352180',
 '3987125231951435424',
 '4033741599548472305',
 '1418137193354531640',
 '3357197785218509494',
 '-6535437352408475447',
 '-7004734840398964545',
 '7704053979528747207',
 '-6378513022733673487',
 '-8716739262777106136',
 '1465521168294687740',
 '383188332671341621',
 '-5390270947453055029',
 '-7811056010021023549',
 '-7026292704542519105',
 '26460100276487942',
 '-2723957341197782107',
 '-4175092425154624123']